In [1]:
import pandas as pd
import numpy as np

### Загружаем последний файл с базой

In [2]:
df = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\База декабрь\База 12.xlsx")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89030 entries, 0 to 89029
Data columns (total 38 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Дата обновления                 89030 non-null  datetime64[ns]
 1   Название проекта                89030 non-null  object        
 2   На англ                         19769 non-null  object        
 3   Промзона                        66038 non-null  object        
 4   Местоположение                  77376 non-null  object        
 5   Метро                           77376 non-null  object        
 6   Расстояние до метро, км         77376 non-null  object        
 7   Время до метро, мин             77376 non-null  object        
 8   Мцк/мцд/бкл                     77376 non-null  object        
 9   Расстояние до мцк/мцд, км       77224 non-null  object        
 10  Время до мцк/мцд, мин           75730 non-null  object        
 11  Бк

### Загружаем предыдущий файл с базой

In [8]:
df2 = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\Новая папка (2)\07.2025_рынок.xlsx", sheet_name = 'массив')

### В последней базе оставляем только последний месяц, если есть лишние

In [10]:
df['Дата обновления'] = pd.to_datetime(df['Дата обновления'], errors='coerce')

In [11]:
df = df[df['Дата обновления'].dt.month == 8]

### Чистим в последней базе сроки сдачи и стадии

In [3]:
df['Срок сдачи'] = np.nan
df['Стадия строительной готовности'] = np.nan

### В столбце Корпус, если номер корпуса идёт в скобках, то удаляем всё за скобками, оставляем только то, что в скобках. Если в скобках есть слово очередь, то ничего не трогаем

In [4]:
df['Корпус'] = df['Корпус'].astype(str)

# Выделим текст в скобках
bracket_content = df['Корпус'].str.extract(r'\((.*?)\)', expand=False)

# Маска: строка содержит скобки
has_brackets = df['Корпус'].str.contains(r'\(.*?\)', na=False)

# Маска: в строке есть слово "очередь" где угодно (внутри или снаружи скобок)
contains_ochered = df['Корпус'].str.contains(r'очередь', case=False, na=False)

# Маска: есть скобки, но НЕТ "очередь" вообще
mask = has_brackets & ~contains_ochered

# Заменяем только те строки, которые соответствуют маске
df.loc[mask, 'Корпус'] = bracket_content[mask].str.strip()

In [ ]:
df2['Корпус'] = df2['Корпус'].astype(str)

# Выделим текст в скобках
bracket_content = df2['Корпус'].str.extract(r'\((.*?)\)', expand=False)

# Маска: строка содержит скобки
has_brackets = df2['Корпус'].str.contains(r'\(.*?\)', na=False)

# Маска: в строке есть слово "очередь" где угодно (внутри или снаружи скобок)
contains_ochered = df2['Корпус'].str.contains(r'очередь', case=False, na=False)

# Маска: есть скобки, но НЕТ "очередь" вообще
mask = has_brackets & ~contains_ochered

# Заменяем только те строки, которые соответствуют маске
df2.loc[mask, 'Корпус'] = bracket_content[mask].str.strip()

### Ещё раз приводим корпуса к текстовому типу

In [5]:
df['Корпус'] = df['Корпус'].astype(str)
df2['Корпус'] = df2['Корпус'].astype(str)

### Меняем названия столбцов

In [15]:
df2.columns = df2.columns.str.capitalize().str.strip()

### Продолжаем чистить Корпуса, меняя все пропуски на '1'

In [6]:
# 1. Удаляем текст 'Жилой дом № ' (всё равно на NaN это не повлияет)
df['Корпус'] = df['Корпус'].replace('Жилой дом № ', '', regex=True)

# Заменяем строку 'nan' и пустые строки на np.nan
df['Корпус'] = df['Корпус'].replace(['nan', r'^\s*$'], np.nan, regex=True)

# 3. Заполняем NaN единицами
df['Корпус'] = df['Корпус'].fillna('1')

In [17]:
# 1. Удаляем текст 'Жилой дом № ' (всё равно на NaN это не повлияет)
df2['Корпус'] = df2['Корпус'].replace('Жилой дом № ', '', regex=True)

# Заменяем строку 'nan' и пустые строки на np.nan
df2['Корпус'] = df2['Корпус'].replace(['nan', r'^\s*$'], np.nan, regex=True)

# 3. Заполняем NaN единицами
df2['Корпус'] = df2['Корпус'].fillna('1')

### Соединяем текущий и предыдущие месяца в один датафрейм

In [36]:
df_new = pd.concat([df2, df], ignore_index=True)

### Заполняем все пропущенные значения нового датафрейма

In [37]:
# Столбцы, в которых нужно заполнить пропуски
columns_to_fill = ['На англ',
 'Промзона',
 'Местоположение',
 'Метро',
 'Расстояние до метро, км',
 'Время до метро, мин',
 'Мцк/мцд/бкл',
 'Расстояние до мцк/мцд, км',
 'Время до мцк/мцд, мин',
 'Бкл',
 'Расстояние до бкл, км',
 'Время до бкл, мин',
 'Статус',
 'Старт',
 'Комментарий',
 'Округ',
 'Район',
 'Адрес',
 'Эскроу',
 'Конструктив',
 'Класс',
 'Договор']

columns_to_fill_by_corpus = ['Срок сдачи', 'Стадия строительной готовности', 'Старый срок сдачи']

df_new = df_new.sort_values(by=[])
df_new = df_new.sort_values(by=['Дата обновления', 'Название проекта', 'Девелопер', 'Корпус'])


# Группируем по названию проекта и заполняем только нужные столбцы
for col in columns_to_fill:
    df_new[col] = df_new.groupby(['Название проекта', 'Девелопер'])[col].ffill()

for col in columns_to_fill_by_corpus:
    df_new[col] = df_new.groupby(['Название проекта', 'Девелопер', 'Корпус'])[col].ffill()

### Сохраняем в новый файл

In [ ]:
df_new = df_new[df_new['Дата обновления'].dt.month.isin([6, 7])]

In [7]:
df.to_excel(r"C:\Users\m.olshanskiy\Desktop\Ноябрь\База4.xlsx", index=False)